# PRUEBAS INTERVALOS DE CONFIANZA

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import mean_absolute_error,max_error,mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.regression.linear_model as linear_model
from statsmodels.tools.tools import pinv_extended  
from scipy import stats
import joblib


In [2]:
# Funciones
# Funciones para el calculo de la regresion Lineal
class LinearRegession():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO.values
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        #r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)
        r_2 = r2_score(Y, yhat)
    
        
        return [model, r_2, yhat]


# Funciones para el calculo de la regresion Lineal
class LinearRegessionv2():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO.values
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        #r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)
        r_2 = r2_score(Y, yhat)
        coef = model.coef_
        intercep = model.intercept_
        residuals = np.power((Y-yhat),2)
        ssr = sum(residuals)

        return  r_2, coef, intercep,ssr,residuals
    

# Funciones apra el calculo de los Intervalos de Confianza
"""
model: Modelo entrenado
x: Valor a testear (X_test)
varianza residual: 
"""
def predict_with_confidence_intervals(model, X, residual_variance, confidence=0.95):
    # Predicciones
    preds = model.predict(X)

    # Grados de libertad
    n = X.shape[0]
    print("n...",n)
    p = X.shape[1]
    print("p....",p)
    df = n - p - 1

    
    # Error estándar de las predicciones
    se_pred = np.sqrt(residual_variance * (1 + np.sum((X - np.mean(X, axis=0))**2 / np.var(X, axis=0), axis=1)))
    print("Error Estandar", se_pred)
    # Valor crítico para el intervalo de confianza
    t_value = stats.t.ppf((1 + confidence) / 2., df)
    print("t_value: ", t_value)
    # Intervalos de confianza
    ci_upper = preds + t_value * se_pred
    ci_lower = preds - t_value * se_pred

    return preds, ci_lower, ci_upper


def cargarGrupos():
    # Cargamos los respectivos grupos
    listaGruposDefinitivos = []
    G0 = pd.read_excel("DataOpt/grupo_N0.xlsx")
    G1 = pd.read_excel("DataOpt/grupo_N1.xlsx")
    G2 = pd.read_excel("DataOpt/grupo_N2.xlsx")

    print(f"Longitud G0: ",{len(G0)},"longitud G1: ", {len(G1)} , "Longitud G2: ", {len(G2)})

    # Eliminamos clumnas
    G0 = G0.drop(["Unnamed: 0"], axis=1)
    G1 = G1.drop(["Unnamed: 0"], axis=1)
    G2 = G2.drop(["Unnamed: 0"], axis=1)
    listaGruposDefinitivos.append(G0)
    listaGruposDefinitivos.append(G1)
    listaGruposDefinitivos.append(G2)

    return listaGruposDefinitivos


# Funcion apra calcular la correlación de los grupos Finales
def CalcularCorrelationInitial(gruposDefinitivos):
    correlacionesIniciales=[]
    for i in range(len(gruposDefinitivos)):
        r_2 = LinearRegessionv2(gruposDefinitivos[i], 0.1, 0.97).CalcularModeloLR()
        correlacionesIniciales.append(r_2)
    
    return correlacionesIniciales





# test_clean: df sin rdt_ajustado y ID_LOTE
def PredctionsModels(lista_modelos, test_clean):
    predictions = []
    for i in range(len(lista_modelos)):
        psi_pred = []
        print("Prediciones Modelo ", i)
        for j in range(len(test_clean)):
            pred = lista_modelos[i].predict(test_clean.values[j].reshape(1,-1))
            psi_pred.append(pred[0])
        
        predictions.append(psi_pred)
    #Lista de prediciones asociada c/modelo
    return predictions


# Funcion para Cargar modelos .PKL
def CargueModelos():
    lista_modelos = []
    for i in range(3):
        model = joblib.load(f'DataOpt/IC/modelo_ols_{i}.pkl') # Carga del modelo.
        lista_modelos.append(model)
    print(len(lista_modelos))
    return lista_modelos


def agregarPredicitions(test_original, predictions):
    df0 = test_original.copy()
    df1= test_original.copy()
    df2 = test_original.copy()
    df0.RDT_AJUSTADO = predictions[0]
    df1.RDT_AJUSTADO = predictions[1]
    df2.RDT_AJUSTADO = predictions[2]
    list_dfs = list([df0,df1,df2])
    return list_dfs

In [3]:
def IntervalosConfianza(df_entrenamiento, df_testeo):
    Y_train = df_entrenamiento.RDT_AJUSTADO.values
    X_train= df_entrenamiento.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values
    Y_test =df_testeo.RDT_AJUSTADO.values
    X_test = df_testeo.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values
     
    # Agreago Constantes al modelo
    X_train = sm.add_constant(X_train, prepend=True)
    X_test = sm.add_constant(X_test, prepend=True)
    model = sm.OLS(Y_train, X_train)
    res= model.fit_regularized(method='elastic_net',alpha=0.1, L1_wt=0.97)
    model_fit_regularized = model.fit(params=res.params)

    #print(model_fit_regularized.summary())
    #model_fit_regularized.conf_int(alpha=0.2)
    #print("Inveralos Coeficientes: ", Intervalos)
    predicciones = model_fit_regularized.predict(exog = X_test)
    #print("prediciones: ", predicciones)

    IC = model_fit_regularized.get_prediction(exog = X_test)
    Sumary_IC = IC.summary_frame(alpha=0.05)
    Sumary_IC["Predicciones"] = predicciones 
    df_summary = Sumary_IC[["mean_ci_lower","mean_ci_upper","Predicciones"]]
    #df_summary = Sumary_IC[["obs_ci_lower","obs_ci_upper","Predicciones"]]

    return df_summary


# Funcion para calculas Intervalos Confianza N  grupos
def calculoIntervalosConfianza(grupos_definitivos, test_original):
    lista_df_IC = []
    for i in range(len(grupos_definitivos)):
        IC = IntervalosConfianza(grupos_definitivos[i],test_original)
        lista_df_IC.append(IC)
    return lista_df_IC


# Funcion apra calcular los DF finales asociados asus respectivos IC
lista_aux_var_ic = ["mean_ci_lower","mean_ci_upper"]
#lista_aux_var_ic = ["obs_ci_lower","obs_ci_upper"]
def DfWithConfidenceIntervals(listaIntervalosConfianza, test_original,lista_aux_var_ic):
    lista_df_ic = []
    for i in range(len(listaIntervalosConfianza)):
        print("Grupo: ", i)
        for j in range(len(lista_aux_var_ic)):
            G_IC= test_original.copy()
            G_IC["RDT_AJUSTADO"] = listaIntervalosConfianza[i][lista_aux_var_ic[j]]
            lista_df_ic.append([G_IC,i])
    return lista_df_ic


def GruposFinales(li):
    lista_modificada = []
    for i in range(len(li)):
        if li[i] == 0:
            lista_modificada.append(0)
        if li[i] == 1:
            lista_modificada.append(0)
        if li[i] == 2:
            lista_modificada.append(1)
        if li[i] == 3:
            lista_modificada.append(1)
        if li[i] == 4:
            lista_modificada.append(2)
        if li[i] == 5:
            lista_modificada.append(2)
    return lista_modificada



def LinearRegessionOLMS(df_entrenamiento):
    Y = df_entrenamiento.RDT_AJUSTADO.values
    X = df_entrenamiento.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values     
    # Agreago Constantes al modelo
    X_train = sm.add_constant(X, prepend=True)
    model = sm.OLS(Y, X_train)
    res= model.fit_regularized(method='elastic_net',alpha=0.1, L1_wt=0.97)
    model_fit_regularized = model.fit(params=res.params)
    r_2 = model_fit_regularized.rsquared
    #print("Nuevo r_2: ", r_2)
    return r_2, model_fit_regularized


def CalcularCorrelationInitialOLS(gruposDefinitivos):
    correlacionesIniciales=[]
    for i in range(len(gruposDefinitivos)):        
        r_2, model = LinearRegessionOLMS(gruposDefinitivos[i])
        # Guardo Correlacion 
        correlacionesIniciales.append(r_2)
        # Gurado modelos
        name_model = f'DataOpt/IC/modelo_ols_{i}.pkl'
        joblib.dump(model,name_model) # Guardo el modelo.  
    return correlacionesIniciales






def metricasPerformanceCLR(y_true, yhat_test):
    R2 = r2_score(y_true, yhat_test)
    MAE =  mean_absolute_error(y_true, yhat_test)
    ME = max_error(y_true, yhat_test)
    MSE = mean_squared_error(y_true, yhat_test)
    print("Coeficiente de determinación : ", R2)
    print("Error absoluto medio: ", MAE)
    print("Maximo error residual: ", ME)
    print("RMSE: ", MSE)


In [17]:
# Retorna el grupo a modificar
def compareCorrelation(CorelacionGruposCalidad, nuevaCorrelation):
    #print("lista grupos: ",lista_grupos)
    arr =  np.array(CorelacionGruposCalidad) - np.array(nuevaCorrelation)
    print("Diferencia: ", arr)
    position = np.where(arr == np.amin(arr))
    #print("Posision grupo: ",position)

    return position


'''
test_original_pred: test original añadido al columna RDT_AJUSTADO promedio de los 3 grupos
list_corr: Lista de las correlaciones iniciales asociadas a c/d grupo

Salida: Retorna las posiciones del cluster donde hay menso impacto
'''
def prubaAletPredict(test_original_pred,list_corr,grupos_definitivos):
    asig_group = []
    for register in range(len(test_original_pred)):
        new_correlation = []
        for group in range(len(grupos_definitivos)):
            grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)        
            new_r2, Model_olms = LinearRegessionOLMS(grupos_definitivos[group])
            new_correlation.append(new_r2)
        
        # Verificar en que gruo se genera la minima correlación 
        pos = compareCorrelation(list_corr, new_correlation)
        print("Posición: ", pos[0])
        asig_group.append(pos[0][0])

        # Eliminamos los registros del grupo

        for i in range(len(grupos_definitivos)):
            grupos_definitivos[i].drop([len(grupos_definitivos[i]) -1 ],axis=0, inplace=True)
    
    return asig_group
    

In [4]:
# Lectura del conjunto de test
test = pd.read_csv("DataOpt/dataset_test_Original.csv")
test.head(5)
test_original = test.copy()
test_clean = test.copy()
test_clean = test.drop(["ID_LOTE","RDT_AJUSTADO"], axis=1)
print("Dimenciones conjunto de test: ", test_clean.shape)
test_clean.head(5)

Dimenciones conjunto de test:  (40, 174)


,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,4,47,94,62000,6,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1
1,1,49,84,60000,17,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,5,46,84,73000,6,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,5,50,76,62000,10,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1
4,4,45,86,56000,9,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [5]:
# 1. Cargue Grupos Definitivos
grupos_definitivos = cargarGrupos()
# 2. Calculo de correlaciones
#list_corr = CalcularCorrelationInitial(grupos_definitivos)
#print("Correlaciones Iniciales: ", list_corr)
 

Longitud G0:  {264} longitud G1:  {260} Longitud G2:  {235}


In [6]:
# Calculos correlaciones OLMS
lista_corr_olsms = CalcularCorrelationInitialOLS(grupos_definitivos)
print("lista Correlaciones Iniciales: ", lista_corr_olsms)

lista Correlaciones Iniciales:  [0.9535896398727264, 0.9506674486861081, 0.9456447959278178]


In [7]:
#3. cargamos los modelos 
list_modelos = CargueModelos()
#4. Obtener prediciones del conjunto de test con c/d modelo
#predictions_models = PredctionsModels(list_modelos, test_clean)
#5. Unir prediciones DF testeo
#dfs_predictions = agregarPredicitions(test_original,predictions_models)


3


In [40]:
predictions_groups = calculoIntervalosConfianza(grupos_definitivos, test_original)

In [41]:
# Definición predicciónes c/u de los grupos
pm1 = predictions_groups[0].Predicciones
pm2 = predictions_groups[1].Predicciones
pm3 = predictions_groups[2].Predicciones

In [42]:
df_predictions = pd.concat([pm1, pm2,pm3], axis=1)

In [43]:
# Unimos todas la predicciones en un unico dataframe

df_predictions.Predicciones

,Predicciones,Predicciones,Predicciones
0,6283.120980,4688.310140,8371.197827
1,9344.091049,6441.959195,7499.793142
2,5332.928757,5894.200786,6144.010168
3,4654.347911,5368.834722,5496.254588
4,4158.518555,6283.379296,3776.499912
5,7649.482044,7394.685097,4990.973995
6,2638.299930,5673.197849,3424.575583
7,6090.855162,7867.484809,6882.473378
8,3813.510257,5512.687048,6057.006628
9,4836.713805,3920.394292,4698.341716


In [45]:

import  random

def buidlDfAleatorios(df_predictions):
    df_aleatorios = pd.DataFrame(columns=['P1', 'P2', 'P3','P4','P5','P6','P7','P8','P9','P10'
                            ,'P11','P12','P13','P14','P15','P16','P17','P18','P19','P20'],
                    index=range(len(df_predictions)))

    for i in range(len(df_predictions)):
        numeros_aleatorios = [random.uniform(df_predictions.iloc[i].min(),df_predictions.iloc[i].max()) for _ in range(20)]
        df_aleatorios.iloc[i] = numeros_aleatorios

    return df_aleatorios

In [50]:
df_aleatorios = buidlDfAleatorios(df_predictions)
df_aleatorios

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20
0,6560.256571,5166.158058,5405.484866,7440.924893,4858.71973,6913.459266,6001.584204,8242.323826,5996.093449,7891.263163,5601.831562,8295.284224,4949.106622,7797.782892,7954.903018,4958.543074,6424.584752,7446.51278,5551.16499,6761.955965
1,7212.330731,7662.528857,9210.498125,9003.761907,7582.242171,6980.267543,8360.351679,7134.093434,8736.269344,8277.437802,8063.85862,8859.720666,7679.046797,7415.48217,9257.822081,6456.36881,6559.454121,6959.459586,7727.773306,9333.794285
2,5680.941513,5704.87234,5860.514834,5375.841965,5481.254658,5496.846729,5586.0695,6044.919029,5674.050212,5712.501345,5545.495132,6087.503794,6118.035531,5587.96223,6033.200013,5596.090285,6116.928046,5397.760024,5429.270507,5929.038699
3,4860.433376,5413.996142,4809.961072,5258.547639,5391.667341,5050.863134,5484.803496,5359.177056,5148.219079,5359.608206,5287.599602,4953.851272,5329.238032,5256.551786,4943.87408,5357.959559,5430.22215,5099.052636,4917.231825,5373.359406
4,5183.292506,3855.652749,4764.209011,6056.949683,5008.500144,4028.853532,5859.161515,5948.531544,6212.623201,6086.08643,6007.152994,6001.432466,5161.269372,5358.384093,4655.769725,5987.131547,5679.820552,4745.608854,5140.589887,6170.560367
5,5413.906819,5457.425547,6348.290371,6836.146811,5724.157707,6598.291506,6027.185921,7582.563838,5883.775348,7616.45501,5461.40688,6939.370854,5563.736578,6930.159899,5910.389276,7482.203366,6396.842655,7467.92525,6259.807801,6113.85406
6,5520.440703,2903.326749,3095.400969,3856.868139,5324.305888,3714.992779,3022.773572,4561.79892,4686.47897,5031.394934,5613.871832,5133.927207,4974.888357,5356.647093,4572.546881,2949.615834,4791.873939,2802.271094,2899.825064,2904.728829
7,6477.041185,6584.819645,6746.870855,7247.226718,6866.299199,7687.541554,6782.143087,6854.558109,6250.324014,7402.456649,6770.489306,6490.776038,7606.264613,7048.214301,6366.632981,6949.379532,6694.485769,6820.119446,6923.426766,7407.977666
8,5328.083389,5421.874875,4418.668309,4984.432168,4557.85256,4935.400732,5656.252357,3961.937853,4010.082293,4719.752923,5753.232916,5719.098146,5335.408361,5332.060567,3940.509025,5625.26353,4072.742927,5032.09607,3981.190351,4126.822077
9,4431.383175,4330.67241,4791.164269,4680.666022,4036.420812,4022.044698,4397.167524,4359.489259,4836.36775,4009.561357,4792.57506,4446.229718,4003.122652,4262.8284,4172.151874,4810.714617,3981.324545,4189.805408,4704.132403,4089.415463


In [51]:

final_grupos = []
for x in range(20):
    test_aleatorios = test_original.copy()
    column= f"P{(x+1)}"
    test_aleatorios.RDT_AJUSTADO = df_aleatorios[column]
    asigng_grupos = prubaAletPredict(test_aleatorios,lista_corr_olsms,grupos_definitivos)
    final_grupos.append(asigng_grupos)

C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)


Diferencia:  [-7.59945934e-05  7.98340902e-03  2.78111111e-03]
Posición:  [0]


C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)


Diferencia:  [ 0.00442221  0.00625243 -0.0005891 ]
Posición:  [2]


C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)


...

Diferencia:  [ 3.26793329e-04  6.75168357e-03 -3.55278940e-05]
Posición:  [2]


C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)


Diferencia:  [0.00042387 0.00724988 0.00043192]
Posición:  [0]


C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_11120\1617319187.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grupos_definitivos[group] = grupos_definitivos[group].append(test_original_pred.loc[register], ignore_index = True)


Diferencia:  [0.00596456 0.0136164  0.0020578 ]
Posición:  [2]


In [52]:
resultados_finales = pd.DataFrame(final_grupos)

In [56]:
resultados_finales

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0,2,2,0,0,2,2,2,2,2,...,2,0,2,2,2,2,2,2,0,0
1,0,2,2,2,2,2,2,2,2,2,...,2,2,2,0,2,2,2,2,2,0
2,0,0,2,0,0,2,2,2,0,2,...,2,0,0,0,2,2,2,2,2,2
3,2,0,0,2,0,0,2,2,2,2,...,2,0,2,0,2,2,2,2,0,2
4,0,2,0,2,0,2,2,2,0,2,...,2,0,2,0,2,2,0,2,0,2
5,0,2,0,2,2,0,2,2,2,2,...,0,0,0,0,2,2,0,2,2,0
6,0,2,0,2,0,2,2,2,2,2,...,2,0,2,0,2,2,0,2,2,2
7,2,2,2,2,0,0,2,2,0,2,...,2,0,2,0,2,2,2,2,0,0
8,0,0,2,2,0,2,2,2,0,2,...,2,2,0,0,2,2,2,2,2,0
9,2,2,2,2,0,0,2,2,0,2,...,0,0,2,2,2,2,0,2,2,0


In [82]:
resultados_finales[16].value_counts()

2    10
0    10
Name: 16, dtype: int64